# Projet TAL

In [25]:
#!pip install torch
#!pip install nltk==3.8.1

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.manifold import TSNE
nltk.download('stopwords')
nltk.download('punkt')
import plotly.express as px

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Yannis_CHUPIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Yannis_CHUPIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

In [26]:
from transformers import BertModel, BertTokenizer
import torch

In [49]:
train_data = pd.read_csv('./sample_data/train.csv',encoding="utf-8")
test_data = pd.read_csv('./sample_data/test.csv')
stemmer = SnowballStemmer('french')
stopwords = set(stopwords.words('french'))
print(train_data.head())
print(train_data.shape,test_data.shape)

AttributeError: 'set' object has no attribute 'words'

In [ ]:
train_data.dropna(axis=0,inplace=True)
test_data.dropna(axis=0,inplace=True)
text_data = pd.concat([train_data['ingredients'],train_data['recette'],train_data['titre']])
sentences = [simple_preprocess(text) for text in text_data]
data = pd.concat([train_data, test_data])

In [ ]:
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, workers=4)

#word2vec_model.train(sentences, total_epoch=10, work=4)


In [50]:
tfidf_features = TfidfVectorizer().fit_transform([' '.join(sentence) for sentence in sentences])
#lexique = tfidf_features.get_feature_names_out()
tfidf_features

<37419x20773 sparse matrix of type '<class 'numpy.float64'>'
	with 1252704 stored elements in Compressed Sparse Row format>

In [ ]:
# from sklearn.pipeline import FeatureUnion

# # Créer un pipeline pour combiner les features
# features = FeatureUnion([
#     ('tfidf', TfidfVectorizer()),
#     ('word2vec', FunctionTransformer(lambda x: np.array([get_text_vector(text, word2vec_model) for text in x]), validate=False)),
# ])

# # Entraîner le modèle avec les features combinées
# X_train_combined = features.fit_transform(train_data['text']) 

### word 2 vec

In [52]:
def get_text_vector(text, model):
       tokens = simple_preprocess(text)
       vectors = [model.wv[token] for token in tokens if token in model.wv]
       if vectors:
           return np.mean(vectors, axis=0)
       else:
           return np.zeros(model.vector_size)  # Vecteur nul si aucun mot n'est trouvé

def get_combined_vector(ingredients, recette, titre, model):
       ing_vector = get_text_vector(ingredients, model)
       recipe_vector = get_text_vector(recette, model)
       title_vector = get_text_vector(titre, model)

       return ing_vector + recipe_vector + title_vector

In [53]:
print(train_data)

train_data_w= train_data.copy()
test_data_w = test_data.copy()

train_data_w['vector'] = train_data_w.apply(lambda row: get_combined_vector(row['ingredients'], row['recette'], row['titre'], word2vec_model), axis=1)
test_data_w['vector'] = test_data_w.apply(lambda row: get_combined_vector(row['ingredients'], row['recette'], row['titre'], word2vec_model), axis=1)


train_data_w = train_data_w.drop(['ingredients', 'recette', 'titre','difficulte','cout','doc_id'], axis=1)
test_data_w = test_data_w.drop(['ingredients', 'recette', 'titre','difficulte','cout','doc_id'], axis=1)

train_data

                   doc_id                                              titre  \
0      recette_221358.xml  Feuilleté de saumon et de poireau, sauce aux c...   
1       recette_48656.xml                       Cake poulet/moutarde/amandes   
2       recette_30049.xml           Bûche à la truite fumée (7ème rencontre)   
3       recette_71424.xml    Gâteau au yaourt au coco sans huile de laetitia   
4      recette_217204.xml                             Crêpes au canard laqué   
...                   ...                                                ...   
12468   recette_88545.xml                           Tarte thon et courgettes   
12469   recette_36440.xml                      Gratin de légumes à la viande   
12470  recette_174633.xml             Tagliatelles fraiches au Montepulciano   
12471   recette_43030.xml  Bavarois praliné pistache, coeur de mousse au ...   
12472   recette_56276.xml                          Grand tralala au chocolat   

                 type             diffi

,doc_id,titre,type,difficulte,cout,ingredients,recette
0,recette_221358.xml,"Feuilleté de saumon et de poireau, sauce aux c...",Plat principal,Facile,Moyen,- 1 gros pavé de saumon - 100 g de crevettes d...,Couper finement le blanc et un peu de vert des...
1,recette_48656.xml,Cake poulet/moutarde/amandes,Entrée,Très facile,Bon marché,- 3 œufs - 150 g de farine - 1 sachet de levur...,"Couper finement l'échalote, la faire revenir à..."
2,recette_30049.xml,Bûche à la truite fumée (7ème rencontre),Entrée,Moyennement difficile,Assez Cher,- 800 g de filet de truite saumonnée fumée en ...,Faites blanchir les épinards à l'eau bouillant...
3,recette_71424.xml,Gâteau au yaourt au coco sans huile de laetitia,Dessert,Très facile,Bon marché,- 1 pot de yaourt - 1 pot de lait de coco - 3 ...,Mélanger dans l'ordre tous les ingrédients en ...
4,recette_217204.xml,Crêpes au canard laqué,Entrée,Moyennement difficile,Moyen,- 90 g de farine - 45 g de maïzena - 2 œufs - ...,"Fouetter les œufs avec l'eau, le lait et le su..."
...,...,...,...,...,...,...,...
12468,recette_88545.xml,Tarte thon et courgettes,Plat principal,Facile,Bon marché,- 1 pâte feuilleté - 2 œufs - 2 courgettes moy...,"Couper les courgettes en fines rondelles, les ..."
12469,recette_36440.xml,Gratin de légumes à la viande,Plat principal,Facile,Bon marché,- 500 g de pommes de terre - 400 g de carotte...,Epluchez les pommes de terre et lavez-les. Cou...
12470,recette_174633.xml,Tagliatelles fraiches au Montepulciano,Plat principal,Difficile,Moyen,- Pour les tagliatelles : - 150 g de semoule f...,Je conseille de préparer les tagliatelles la v...
12471,recette_43030.xml,"Bavarois praliné pistache, coeur de mousse au ...",Dessert,Difficile,Assez Cher,- Pour le biscuit Dacquoise : - 160 g de blanc...,Petits commentaires :J'ai indiqué les recettes...


### BERT

In [54]:
# Charger le tokenizer et le modèle BERT pré-entraîné
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_text_vector_bert(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state
    text_vector = torch.mean(embeddings, dim=1).squeeze()
    return text_vector.numpy()

def get_combined_vector_bert(ingredients, recette, titre, model, tokenizer):
    # Obtenir les vecteurs pour chaque champ de texte
    vector_ingredients = get_text_vector_bert(ingredients, model, tokenizer)
    vector_recette = get_text_vector_bert(recette, model, tokenizer)
    vector_titre = get_text_vector_bert(titre, model, tokenizer)

    # Combiner les vecteurs (par exemple, en les additionnant)
    combined_vector = vector_ingredients + vector_recette + vector_titre

    return combined_vector


In [56]:
print(train_data)

train_data_b= train_data.copy()
test_data_b = test_data.copy()

train_data_b['vector'] = train_data_b.apply(lambda row: get_combined_vector_bert(row['ingredients'], row['recette'], row['titre'], model, tokenizer), axis=1)
test_data_b['vector'] = test_data_b.apply(lambda row: get_combined_vector_bert(row['ingredients'], row['recette'], row['titre'], model, tokenizer), axis=1)

# Supprimer les colonnes de texte
train_data_b = train_data_b.drop(['ingredients', 'recette', 'titre', 'difficulte', 'cout', 'doc_id'], axis=1)
test_data_b = test_data_b.drop(['ingredients', 'recette', 'titre', 'difficulte', 'cout', 'doc_id'], axis=1)

# Afficher les données transformées
print(train_data)
print(test_data)

                   doc_id                                              titre  \
0      recette_221358.xml  Feuilleté de saumon et de poireau, sauce aux c...   
1       recette_48656.xml                       Cake poulet/moutarde/amandes   
2       recette_30049.xml           Bûche à la truite fumée (7ème rencontre)   
3       recette_71424.xml    Gâteau au yaourt au coco sans huile de laetitia   
4      recette_217204.xml                             Crêpes au canard laqué   
...                   ...                                                ...   
12468   recette_88545.xml                           Tarte thon et courgettes   
12469   recette_36440.xml                      Gratin de légumes à la viande   
12470  recette_174633.xml             Tagliatelles fraiches au Montepulciano   
12471   recette_43030.xml  Bavarois praliné pistache, coeur de mousse au ...   
12472   recette_56276.xml                          Grand tralala au chocolat   

                 type             diffi

KeyboardInterrupt: 

In [ ]:
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)
print(train_data.shape,val_data.shape,test_data.shape)

(9978, 2) (2495, 2) (1388, 2)


In [ ]:
def preprocess_text(text):
  tokens = nltk.word_tokenize(text)
  tokens = [stemmer.stem(token.lower()) if token.lower() not in stopwords else token.lower() for token in tokens]
  return "".join(tokens)

In [ ]:
X_train = np.array(train_data['vector'].tolist())
y_train = np.array(train_data['type'].tolist())
X_val = np.array(val_data['vector'].tolist())
y_val = np.array(val_data['type'].tolist())
X_test = np.array(test_data['vector'].tolist())
y_test = np.array(test_data['type'].tolist())
"""
from sklearn.decomposition import PCA

   pca = PCA(n_components=0.95)  # Conserver 95 % de la variance
   X_train_combined_pca = pca.fit_transform(X_train_combined)
   X_val_combined_pca = pca.transform(X_val_combined)
   X_test_combined_pca = pca.transform(X_test_combined)

   svc_model_combined.fit(X_train_combined_pca, y_train)"""

'\nfrom sklearn.decomposition import PCA\n\n   pca = PCA(n_components=0.95)  # Conserver 95 % de la variance\n   X_train_combined_pca = pca.fit_transform(X_train_combined)\n   X_val_combined_pca = pca.transform(X_val_combined)\n   X_test_combined_pca = pca.transform(X_test_combined)\n\n   svc_model_combined.fit(X_train_combined_pca, y_train)'

In [ ]:
X_train_tfidf = tfidf_features[:len(train_data)].toarray()
X_val_tfidf = tfidf_features[len(train_data):len(train_data) + len(val_data)].toarray()
X_test_tfidf = tfidf_features[len(train_data) + len(val_data):len(train_data) + len(val_data) + len(test_data)].toarray()

X_train_combined = np.concatenate((X_train, X_train_tfidf), axis=1)
X_val_combined = np.concatenate((X_val, X_val_tfidf), axis=1)
X_test_combined = np.concatenate((X_test, X_test_tfidf), axis=1)

## Modèle

In [ ]:
# Entraîner le modèle SVC
svc_model = SVC(kernel='linear')
svc_model.fit(X_train, y_train)

# Entraîner le modèle Naive Bayes
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

# Entraîner le modèle Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [ ]:
# Entraîner le modèle de régression logistique
rl_model = LogisticRegression()
rl_model.fit(X_train, y_train)

c:\Users\Yannis_CHUPIN\Documents\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



LogisticRegression()

In [ ]:
# Faire des prédictions
y_pred_svc = svc_model.predict(X_test)
y_pred_nb = nb_model.predict(X_test)
y_pred_rf = rf_model.predict(X_test)
y_pred_rl = rl_model.predict(X_test)

### Métriques

In [ ]:
# Calculer les métriques
print("SVC:")
print(classification_report(y_test, y_pred_svc))

print("Naive Bayes:")
print(classification_report(y_test, y_pred_nb))

print("Random Forest:")
print(classification_report(y_test, y_pred_rf))

print("Logistical regression:")
print(classification_report(y_test, y_pred_rl))

SVC:
                precision    recall  f1-score   support

       Dessert       0.98      0.99      0.99       407
        Entrée       0.74      0.57      0.64       337
Plat principal       0.80      0.90      0.85       644

      accuracy                           0.84      1388
     macro avg       0.84      0.82      0.83      1388
  weighted avg       0.84      0.84      0.84      1388

Naive Bayes:
                precision    recall  f1-score   support

       Dessert       0.94      0.99      0.96       407
        Entrée       0.59      0.65      0.62       337
Plat principal       0.83      0.76      0.79       644

      accuracy                           0.80      1388
     macro avg       0.79      0.80      0.79      1388
  weighted avg       0.81      0.80      0.80      1388

Random Forest:
                precision    recall  f1-score   support

       Dessert       0.95      0.98      0.97       407
        Entrée       0.75      0.51      0.61       337
Plat pri

### Les matrices de confusion :

In [ ]:
encoder = LabelEncoder()
encoder.fit(y_train)

def plot_confusion_matrix(y_test, y_pred, title):
    cm = confusion_matrix(y_test, y_pred)
    fig = px.imshow(cm, text_auto=True, labels=dict(y="True Label", x="Predicted Label"),
                    x=encoder.classes_, y=encoder.classes_, title=title)
    fig.show()

# Générer et visualiser les matrices de confusion
plot_confusion_matrix(y_test, y_pred_svc, "SVC Confusion Matrix")
plot_confusion_matrix(y_test, y_pred_nb, "Naive Bayes Confusion Matrix")
plot_confusion_matrix(y_test, y_pred_rf, "Random Forest Confusion Matrix")
plot_confusion_matrix(y_test, y_pred_rl, "Logistical Regression Confusion Matrix")
